In [ ]:
# default_exp core.scraping.linescore

# Linescore

> Scrapes CZ linescore information.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

from czapi.core.scraping.base import *
from bs4 import BeautifulSoup, Tag
from collections import defaultdict
from typing import List

In [ ]:
#export
def generate_dict_from_table(

    table : Tag

):
    d = defaultdict(list)
    team = None
    
    # TODO : add error handling for when no table is passed / None
    
    for tag in table.find_all('td'):
        if tag.attrs.get('class') == ['linescoreteam']:
            team = tag.a.string
            d[team] = defaultdict(list)
            d[team]['href'] = tag.a['href']
        elif tag.attrs.get('class') == ['linescorehammer']:
            d[team]['hammer'] = not bool(tag.string)
        elif tag.attrs.get('class') == ['linescoreend']:
            d[team]['score'].append(tag.string.strip())
        elif tag.attrs.get('class') == ['linescorefinal']:
            d[team]['finalscore'] = tag.b.string.strip()
        
    return d

In [ ]:
# export


def get_table_from_index(

     tables : List[Tag]
    ,game_number : int

):
    # TODO confirm this is the kind of error handling we want
    if game_number < 1 :
        raise ValueError('Table number must be greater than 0.')
        
    game_idx = game_number -1 
        
    # TODO add error handling for when index is out of range
    
    try:
        return tables[game_idx]
    except IndexError as e:
        raise IndexError(". ".join([str(e),"Are you sure that game number is valid?"]))
    
def get_boxscore(

     cz_event_id : str
    ,cz_draw_id : int
    ,game_number : int 
    ,**request_kwargs
):
    url = 'https://curlingzone.com/event.php?eventid=%s&view=Scores&showdrawid=%s#1'%(cz_event_id,cz_draw_id)
    soup = make_soup(url=url,**request_kwargs)
    
    return _get_boxscore(
    
         soup = soup
        ,game_number = game_number
    
    )


def _get_boxscore(

     soup : BeautifulSoup
    ,game_number : int
):
    
    tables = soup.find_all('table',attrs={'class':'linescorebox'})
    
    return __get_boxscore(
        
         tables = tables
        ,game_number = game_number
    
    )
    


def __get_boxscore(

     tables : List[Tag]
    ,game_number : int

):
    
    try:
        table = get_table_from_index(

             tables = tables
            ,game_number = game_number
        )

    except IndexError as e:
        # TODO : change return value based on what makes sense for the API
        return {}
        
    return generate_dict_from_table(
    
        table = table
    
    )

    
    
    

In [ ]:
d = {
    
    'Wayne Tuck Jr.' : {
        
         'href' : 'event.php?view=Team&eventid=6400&teamid=144353&profileid=12486#1'
        ,'hammer' : True
        ,'score' : ['0','2','0','0','0','0','1','1','1','0']
        ,'finalscore' : '5'
        
    }
    ,'Matthew Hall' : {
        
         'href' : 'event.php?view=Team&eventid=6400&teamid=144347&profileid=12435#1'
        ,'hammer' : False
        ,'score' : ['0','0','4','0','0','1','0','0','0','2']
        ,'finalscore' : '7'
    }
    
}

assert get_boxscore(

     cz_event_id = 6400
    ,cz_draw_id = 2
    ,game_number = 1

) == d

assert get_boxscore(

     cz_event_id = 70000
    ,cz_draw_id = 1
    ,game_number = 1

) == {}